<a href="https://colab.research.google.com/github/ChaitanyaQR/Job_A_Thon_Supplement_Sales_Prediction/blob/main/Supplement_Sales_Prediction_LSTM_tune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df_train = pd.read_csv('TRAIN.csv')
df_test = pd.read_csv('TEST_FINAL.csv')

In [ ]:
df_train.head()


,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188340 entries, 0 to 188339
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   ID             188340 non-null  object 
 1   Store_id       188340 non-null  int64  
 2   Store_Type     188340 non-null  object 
 3   Location_Type  188340 non-null  object 
 4   Region_Code    188340 non-null  object 
 5   Date           188340 non-null  object 
 6   Holiday        188340 non-null  int64  
 7   Discount       188340 non-null  object 
 8   #Order         188340 non-null  int64  
 9   Sales          188340 non-null  float64
dtypes: float64(1), int64(3), object(6)
memory usage: 14.4+ MB


In [ ]:
df_Store_Type=pd.get_dummies(df_train['Store_Type'],prefix='Store_type',drop_first=False)
df_Location_Type=pd.get_dummies(df_train['Location_Type'],prefix='Location_Type',drop_first=False)
df_Region_Code=pd.get_dummies(df_train['Region_Code'],prefix='Region_Code',drop_first=False)

df_test_Store_Type=pd.get_dummies(df_test['Store_Type'],prefix='Store_type',drop_first=False)
df_test_Location_Type=pd.get_dummies(df_test['Location_Type'],prefix='Location_Type',drop_first=False)
df_test_Region_Code=pd.get_dummies(df_test['Region_Code'],prefix='Region_Code',drop_first=False)

In [ ]:
df_train=pd.concat([df_train,df_Store_Type,df_Location_Type,df_Region_Code],axis=1)
df_test=pd.concat([df_test,df_test_Store_Type,df_test_Location_Type,df_test_Region_Code],axis=1)

In [ ]:
df_train

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales,Store_type_S1,Store_type_S2,Store_type_S3,Store_type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4
0,T1000001,1,S1,L3,R1,2018-01-01,1,Yes,9,7011.84,1,0,0,0,0,0,1,0,0,1,0,0,0
1,T1000002,253,S4,L2,R1,2018-01-01,1,Yes,60,51789.12,0,0,0,1,0,1,0,0,0,1,0,0,0
2,T1000003,252,S3,L2,R1,2018-01-01,1,Yes,42,36868.20,0,0,1,0,0,1,0,0,0,1,0,0,0
3,T1000004,251,S2,L3,R1,2018-01-01,1,Yes,23,19715.16,0,1,0,0,0,0,1,0,0,1,0,0,0
4,T1000005,250,S2,L3,R4,2018-01-01,1,Yes,62,45614.52,0,1,0,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,T1188336,149,S2,L3,R2,2019-05-31,1,Yes,51,37272.00,0,1,0,0,0,0,1,0,0,0,1,0,0
188336,T1188337,153,S4,L2,R1,2019-05-31,1,No,90,54572.64,0,0,0,1,0,1,0,0,0,1,0,0,0
188337,T1188338,154,S1,L3,R2,2019-05-31,1,No,56,31624.56,1,0,0,0,0,0,1,0,0,0,1,0,0
188338,T1188339,155,S3,L1,R2,2019-05-31,1,Yes,70,49162.41,0,0,1,0,1,0,0,0,0,0,1,0,0


In [ ]:
# Import label encoder
from sklearn import preprocessing
 
# label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
df_train['Discount']=label_encoder.fit_transform(df_train['Discount'])
df_train['Discount']
df_test['Discount']=label_encoder.fit_transform(df_test['Discount'])
df_test['Discount']

0        0
1        0
2        0
3        0
4        0
        ..
22260    0
22261    0
22262    1
22263    0
22264    0
Name: Discount, Length: 22265, dtype: int64

In [ ]:
df_train

,ID,Store_id,Store_Type,Location_Type,Region_Code,Date,Holiday,Discount,#Order,Sales,Store_type_S1,Store_type_S2,Store_type_S3,Store_type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4
0,T1000001,1,S1,L3,R1,2018-01-01,1,1,9,7011.84,1,0,0,0,0,0,1,0,0,1,0,0,0
1,T1000002,253,S4,L2,R1,2018-01-01,1,1,60,51789.12,0,0,0,1,0,1,0,0,0,1,0,0,0
2,T1000003,252,S3,L2,R1,2018-01-01,1,1,42,36868.20,0,0,1,0,0,1,0,0,0,1,0,0,0
3,T1000004,251,S2,L3,R1,2018-01-01,1,1,23,19715.16,0,1,0,0,0,0,1,0,0,1,0,0,0
4,T1000005,250,S2,L3,R4,2018-01-01,1,1,62,45614.52,0,1,0,0,0,0,1,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,T1188336,149,S2,L3,R2,2019-05-31,1,1,51,37272.00,0,1,0,0,0,0,1,0,0,0,1,0,0
188336,T1188337,153,S4,L2,R1,2019-05-31,1,0,90,54572.64,0,0,0,1,0,1,0,0,0,1,0,0,0
188337,T1188338,154,S1,L3,R2,2019-05-31,1,0,56,31624.56,1,0,0,0,0,0,1,0,0,0,1,0,0
188338,T1188339,155,S3,L1,R2,2019-05-31,1,1,70,49162.41,0,0,1,0,1,0,0,0,0,0,1,0,0


In [ ]:
df_train.drop(['Store_Type','Location_Type','Region_Code'],inplace=True,axis=1)
df_test.drop(['Store_Type','Location_Type','Region_Code'],inplace=True,axis=1)

In [ ]:
df_train.drop(['ID','Date','#Order','Store_id'],inplace=True,axis=1)
df_test.drop(['ID','Date','Store_id'],inplace=True,axis=1)

In [ ]:
df_train.head()

,Holiday,Discount,Sales,Store_type_S1,Store_type_S2,Store_type_S3,Store_type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4
0,1,1,7011.84,1,0,0,0,0,0,1,0,0,1,0,0,0
1,1,1,51789.12,0,0,0,1,0,1,0,0,0,1,0,0,0
2,1,1,36868.20,0,0,1,0,0,1,0,0,0,1,0,0,0
3,1,1,19715.16,0,1,0,0,0,0,1,0,0,1,0,0,0
4,1,1,45614.52,0,1,0,0,0,0,1,0,0,0,0,0,1


In [ ]:
df_test.head()

,Holiday,Discount,Store_type_S1,Store_type_S2,Store_type_S3,Store_type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4
0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0
1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0
2,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0
3,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1
4,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0


In [ ]:
s = (df_train.dtypes == 'object')
object_cols = list(s[s].index)
print("Categorical variables:")
print(object_cols)

Categorical variables:
[]


In [ ]:
from sklearn.feature_selection import VarianceThreshold
var_thres=VarianceThreshold(threshold=0)

In [ ]:
var_thres.fit(df_train.drop(['Sales'],axis=1))

VarianceThreshold(threshold=0)

In [ ]:
var_thres.get_support()

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True])

In [ ]:
sum(var_thres.get_support())

15

In [ ]:
df_train.drop(['Sales'],axis=1).corr()

,Holiday,Discount,Store_type_S1,Store_type_S2,Store_type_S3,Store_type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4
Holiday,1.000000e+00,0.019545,8.037425e-17,5.077300e-18,-3.888942e-17,7.029552e-17,-1.155020e-16,8.203876e-18,6.366159e-18,1.398366e-17,-3.256673e-17,-2.924524e-17,-2.453558e-17,3.953269e-17,4.771960e-18
Discount,1.954527e-02,1.000000,-1.299490e-03,-1.807669e-03,1.782628e-03,1.624845e-03,-1.657321e-03,1.848451e-03,7.924557e-04,-9.402348e-04,-2.071222e-04,9.431541e-04,-2.441909e-03,1.688978e-03,-1.688930e-04
Store_type_S1,8.037425e-17,-0.001299,1.000000e+00,-4.018837e-01,-3.673470e-01,-5.360756e-01,2.784168e-01,-3.802639e-01,1.301399e-01,4.959404e-02,-1.200205e-01,-2.020225e-01,1.275578e-01,5.786181e-02,3.892305e-02
Store_type_S2,5.077300e-18,-0.001808,-4.018837e-01,1.000000e+00,-1.656555e-01,-2.417437e-01,-2.492020e-01,-2.507229e-01,1.892532e-01,1.886316e-01,4.605906e-01,-1.288032e-01,6.533318e-02,-2.139744e-02,1.178165e-01
Store_type_S3,-3.888942e-17,0.001783,-3.673470e-01,-1.656555e-01,1.000000e+00,-2.209691e-01,7.006360e-02,1.045313e-01,-8.043676e-02,-9.614383e-02,-1.099802e-01,2.898137e-02,-8.610841e-02,7.049625e-02,-1.356528e-02
Store_type_S4,7.029552e-17,0.001625,-5.360756e-01,-2.417437e-01,-2.209691e-01,1.000000e+00,-1.696448e-01,5.702210e-01,-2.468228e-01,-1.403043e-01,-1.604959e-01,3.201478e-01,-1.353497e-01,-1.047891e-01,-1.334564e-01
Location_Type_L1,-1.155020e-16,-0.001657,2.784168e-01,-2.492020e-01,7.006360e-02,-1.696448e-01,1.000000e+00,-5.349415e-01,-3.947951e-01,-2.244179e-01,-2.567146e-01,-5.875165e-02,6.496752e-03,1.457054e-02,5.439067e-02
Location_Type_L2,8.203876e-18,0.001848,-3.802639e-01,-2.507229e-01,1.045313e-01,5.702210e-01,-5.349415e-01,1.000000e+00,-2.559906e-01,-1.455157e-01,-1.664573e-01,8.024347e-02,-8.361394e-02,1.257942e-02,-1.597575e-02
Location_Type_L3,6.366159e-18,0.000792,1.301399e-01,1.892532e-01,-8.043676e-02,-2.468228e-01,-3.947951e-01,-2.559906e-01,1.000000e+00,-1.073928e-01,-1.228481e-01,-4.278727e-02,2.177092e-02,5.887841e-02,-4.239698e-02
Location_Type_L4,1.398366e-17,-0.000940,4.959404e-02,1.886316e-01,-9.614383e-02,-1.403043e-01,-2.244179e-01,-1.455157e-01,-1.073928e-01,1.000000e+00,-6.983192e-02,-2.817683e-02,7.690482e-02,-2.628064e-02,-2.999997e-02


In [ ]:
def correlation(dataset, threshold):
    col_corr = set()  # Set of all the names of correlated columns
    corr_matrix = dataset.corr()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if (corr_matrix.iloc[i, j]) > threshold: # we are interested in absolute coeff value
                colname = corr_matrix.columns[i]  # getting the name of column
                col_corr.add(colname)
    return col_corr

In [ ]:
corr_features = correlation(df_train.drop(['Sales'],axis=1), 0.75)
len(set(corr_features))

0

In [ ]:
from sklearn.preprocessing import StandardScaler,MinMaxScaler

sc=StandardScaler()
sc_mn=MinMaxScaler()

In [ ]:
sc.fit(df_train.drop(['Sales'],axis=1))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [ ]:
scaled_features= sc.transform(df_train.drop(['Sales'],axis=1))
scaled_features
test_scaled_features=sc.transform(df_test)
test_scaled_features

array([[-0.38959686, -0.90004106, -0.94402951, ..., -0.63548891,
         1.80116242, -0.39840954],
       [-0.38959686, -0.90004106,  1.05928892, ..., -0.63548891,
        -0.55519702, -0.39840954],
       [-0.38959686, -0.90004106, -0.94402951, ..., -0.63548891,
        -0.55519702, -0.39840954],
       ...,
       [-0.38959686,  1.11106042,  1.05928892, ..., -0.63548891,
         1.80116242, -0.39840954],
       [-0.38959686, -0.90004106,  1.05928892, ..., -0.63548891,
        -0.55519702,  2.50998008],
       [-0.38959686, -0.90004106, -0.94402951, ...,  1.57359158,
        -0.55519702, -0.39840954]])

In [ ]:
df = pd.DataFrame(scaled_features,columns=df_train.drop(['Sales'],axis=1).columns[:])
df.head()
df_test_final = pd.DataFrame(test_scaled_features,columns=df_test.columns[:])
df_test_final.head()

,Holiday,Discount,Store_type_S1,Store_type_S2,Store_type_S3,Store_type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4
0,-0.389597,-0.900041,-0.944030,-0.425711,-0.389127,1.761001,-0.908295,1.697934,-0.434655,-0.247076,-0.282633,-0.717302,-0.635489,1.801162,-0.39841
1,-0.389597,-0.900041,1.059289,-0.425711,-0.389127,-0.567859,1.100964,-0.588951,-0.434655,-0.247076,-0.282633,1.394112,-0.635489,-0.555197,-0.39841
2,-0.389597,-0.900041,-0.944030,-0.425711,-0.389127,1.761001,-0.908295,1.697934,-0.434655,-0.247076,-0.282633,1.394112,-0.635489,-0.555197,-0.39841
3,-0.389597,-0.900041,1.059289,-0.425711,-0.389127,-0.567859,1.100964,-0.588951,-0.434655,-0.247076,-0.282633,-0.717302,-0.635489,-0.555197,2.50998
4,-0.389597,-0.900041,1.059289,-0.425711,-0.389127,-0.567859,1.100964,-0.588951,-0.434655,-0.247076,-0.282633,-0.717302,1.573592,-0.555197,-0.39841


In [ ]:
df.head()

,Holiday,Discount,Store_type_S1,Store_type_S2,Store_type_S3,Store_type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4
0,2.566756,1.11106,1.059289,-0.425711,-0.389127,-0.567859,-0.908295,-0.588951,2.300675,-0.247076,-0.282633,1.394112,-0.635489,-0.555197,-0.39841
1,2.566756,1.11106,-0.944030,-0.425711,-0.389127,1.761001,-0.908295,1.697934,-0.434655,-0.247076,-0.282633,1.394112,-0.635489,-0.555197,-0.39841
2,2.566756,1.11106,-0.944030,-0.425711,2.569857,-0.567859,-0.908295,1.697934,-0.434655,-0.247076,-0.282633,1.394112,-0.635489,-0.555197,-0.39841
3,2.566756,1.11106,-0.944030,2.349012,-0.389127,-0.567859,-0.908295,-0.588951,2.300675,-0.247076,-0.282633,1.394112,-0.635489,-0.555197,-0.39841
4,2.566756,1.11106,-0.944030,2.349012,-0.389127,-0.567859,-0.908295,-0.588951,2.300675,-0.247076,-0.282633,-0.717302,-0.635489,-0.555197,2.50998


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train=df
X_test=df_test_final
y_train=df_train['Sales']

In [ ]:
X_train.shape

(188340, 15)

In [ ]:
X_train

,Holiday,Discount,Store_type_S1,Store_type_S2,Store_type_S3,Store_type_S4,Location_Type_L1,Location_Type_L2,Location_Type_L3,Location_Type_L4,Location_Type_L5,Region_Code_R1,Region_Code_R2,Region_Code_R3,Region_Code_R4
0,2.566756,1.111060,1.059289,-0.425711,-0.389127,-0.567859,-0.908295,-0.588951,2.300675,-0.247076,-0.282633,1.394112,-0.635489,-0.555197,-0.39841
1,2.566756,1.111060,-0.944030,-0.425711,-0.389127,1.761001,-0.908295,1.697934,-0.434655,-0.247076,-0.282633,1.394112,-0.635489,-0.555197,-0.39841
2,2.566756,1.111060,-0.944030,-0.425711,2.569857,-0.567859,-0.908295,1.697934,-0.434655,-0.247076,-0.282633,1.394112,-0.635489,-0.555197,-0.39841
3,2.566756,1.111060,-0.944030,2.349012,-0.389127,-0.567859,-0.908295,-0.588951,2.300675,-0.247076,-0.282633,1.394112,-0.635489,-0.555197,-0.39841
4,2.566756,1.111060,-0.944030,2.349012,-0.389127,-0.567859,-0.908295,-0.588951,2.300675,-0.247076,-0.282633,-0.717302,-0.635489,-0.555197,2.50998
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188335,2.566756,1.111060,-0.944030,2.349012,-0.389127,-0.567859,-0.908295,-0.588951,2.300675,-0.247076,-0.282633,-0.717302,1.573592,-0.555197,-0.39841
188336,2.566756,-0.900041,-0.944030,-0.425711,-0.389127,1.761001,-0.908295,1.697934,-0.434655,-0.247076,-0.282633,1.394112,-0.635489,-0.555197,-0.39841
188337,2.566756,-0.900041,1.059289,-0.425711,-0.389127,-0.567859,-0.908295,-0.588951,2.300675,-0.247076,-0.282633,-0.717302,1.573592,-0.555197,-0.39841
188338,2.566756,1.111060,-0.944030,-0.425711,2.569857,-0.567859,1.100964,-0.588951,-0.434655,-0.247076,-0.282633,-0.717302,1.573592,-0.555197,-0.39841


In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
X_train.shape[1]

15

In [ ]:
model = keras.Sequential([
                          keras.layers.Dense(512,input_dim = X_train.shape[1],kernel_initializer='normal',activation='relu'),
                          keras.layers.Dense(512,kernel_initializer='normal',activation='relu'),
                          keras.layers.Dense(256,kernel_initializer='normal',activation='relu'),
                          keras.layers.Dense(256,kernel_initializer='normal',activation='relu'),
                          keras.layers.Dense(128,kernel_initializer='normal',activation='relu'),
                          keras.layers.Dense(1,kernel_initializer='normal',activation='linear')

])

In [ ]:
model.compile(loss='mean_absolute_error',optimizer='adam',metrics=['mean_absolute_error'])

In [ ]:
model.fit(X_train,y_train,epochs=20,validation_split=0.3)


Epoch 1/20
4120/4120 [==============================] - 31s 7ms/step - loss: 8708.4629 - mean_absolute_error: 8708.4629 - val_loss: 9290.7363 - val_mean_absolute_error: 9290.7363
Epoch 2/20
4120/4120 [==============================] - 32s 8ms/step - loss: 8283.1396 - mean_absolute_error: 8283.1396 - val_loss: 9199.1357 - val_mean_absolute_error: 9199.1357
Epoch 3/20
4120/4120 [==============================] - 29s 7ms/step - loss: 8268.4375 - mean_absolute_error: 8268.4375 - val_loss: 9285.0312 - val_mean_absolute_error: 9285.0312
Epoch 4/20
4120/4120 [==============================] - 32s 8ms/step - loss: 8243.9912 - mean_absolute_error: 8243.9912 - val_loss: 9168.3027 - val_mean_absolute_error: 9168.3027
Epoch 5/20
4120/4120 [==============================] - 31s 8ms/step - loss: 8251.3662 - mean_absolute_error: 8251.3662 - val_loss: 9423.3213 - val_mean_absolute_error: 9423.3213
Epoch 6/20
4120/4120 [==============================] - 29s 7ms/step - loss: 8221.9639 - mean_absolute_er

In [ ]:
y_pred = model.predict(X_test)
ANN_df=pd.DataFrame(y_pred)

In [ ]:
df_testsample=pd.read_csv('TEST_FINAL.csv')
df_final_ann=pd.concat([df_testsample['ID'],ANN_df],axis=1)
df_final_ann.columns=['ID','Sales']
df_final_ann.to_csv('sample_submission_ann.csv',index=False)

In [ ]:
import pickle
filename = 'SVR_model.pkl'
pickle.dump(svr, open(filename, 'wb'))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')